In [2]:
import tensorflow as tf
import numpy as np
from tensorflow import keras


with np.load('data/mnist_fashion.npz') as data:
    X = data['x_train']
    Y = data['y_train']
    x_test = data['x_test']
    y_test = data['y_test']
    
    
x_valid, x_train = X[:5000]/255.0 , X[5000:]/255.0
y_valid, y_train = Y[:5000] , Y[5000:]

class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
                "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]


model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape = [28,28]))
model.add(keras.layers.Dense(300, activation = 'relu'))
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(10, activation="softmax"))

model.compile(loss="sparse_categorical_crossentropy", optimizer = "sgd", metrics=["accuracy"])

In [8]:
import os
root_log_dir = os.path.join(os.curdir, "my_logs")

def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_log_dir, run_id)

run_logdir = get_run_logdir()

tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In [9]:
history = model.fit(x_train, y_train, epochs=30, validation_data=(x_valid, y_valid), callbacks=[tensorboard_cb])

Train on 55000 samples, validate on 5000 samples
Epoch 1/30
55000/55000 [==============================] - 4s 71us/sample - loss: 0.6165 - accuracy: 0.8399 - val_loss: 0.3135 - val_accuracy: 0.9140
Epoch 2/30
55000/55000 [==============================] - 3s 60us/sample - loss: 0.2885 - accuracy: 0.9174 - val_loss: 0.2405 - val_accuracy: 0.9310
Epoch 3/30
55000/55000 [==============================] - 4s 65us/sample - loss: 0.2368 - accuracy: 0.9324 - val_loss: 0.2043 - val_accuracy: 0.9402
Epoch 4/30
55000/55000 [==============================] - 3s 62us/sample - loss: 0.2031 - accuracy: 0.9418 - val_loss: 0.1798 - val_accuracy: 0.9500
Epoch 5/30
55000/55000 [==============================] - 4s 79us/sample - loss: 0.1780 - accuracy: 0.9494 - val_loss: 0.1576 - val_accuracy: 0.9564
Epoch 6/30
55000/55000 [==============================] - 4s 80us/sample - loss: 0.1571 - accuracy: 0.9548 - val_loss: 0.1438 - val_accuracy: 0.9598
Epoch 7/30
55000/55000 [==============================] -

In [12]:
test_logdir = get_run_logdir()
writer = tf.summary.create_file_writer(test_logdir)

with writer.as_default():
    for step in range(1, 1000 + 1):
        tf.summary.scalar("my_scalar", np.sin(step / 10), step=step)
        data = (np.random.randn(100) + 2) * step / 100 # some random data
        tf.summary.histogram("my_hist", data, buckets=50, step=step)
        images = np.random.rand(2, 32, 32, 3) # random 32×32 RGB images
        tf.summary.image("my_images", images * step / 1000, step=step)
        texts = ["The step is " + str(step), "Its square is " + str(step**2)]
        tf.summary.text("my_text", texts, step=step)
        sine_wave = tf.math.sin(tf.range(12000) / 48000 * 2 * np.pi * step)
        audio = tf.reshape(tf.cast(sine_wave, tf.float32), [1, -1, 1])
        tf.summary.audio("my_audio", audio, sample_rate=48000, step=step)